# Yahoo Finance Python API Demo

## Import

Docs can be found at https://pypi.org/project/yfinance/

In [ ]:
import yfinance as yf
import requests
import pandas as pd

## Using Ticker

In [ ]:
apple_ticker = yf.Ticker("AAPL")

### History

In [ ]:
hist = apple_ticker.history(period="max")

In [ ]:
hist

In [ ]:
hist['Open'].plot()

### Earnings

In [ ]:
apple_ticker.earnings

In [ ]:
apple_ticker.earnings.plot()

## ISIN and Ticker

#### Ticker --> ISIN:

In [ ]:
apple_ticker.isin

#### ISIN --> Ticker ???

In [ ]:
def isin_request(isin):
    url = "https://query2.finance.yahoo.com/v1/finance/search"
    params = {'q': isin, 'quotesCount': 1, 'newsCount': 0}
    r = requests.get(url, params=params)
    data = r.json()
    if len(data['quotes']) != 0:
        symbol = data['quotes'][0]['symbol']
        ticker = yf.Ticker(symbol)
        if ticker is not None:
            output = {
                'symbol': symbol,
                'info': ticker.info
            }
    else:
        output = {
            'symbol': None,
            'info': {}
        }
    return output

In [ ]:
query_response = isin_request('AU0000066086')

In [ ]:
query_response

In [ ]:
isin_info = query_response['info']

In [ ]:
def isin_to_ticker_symbol(isin):
    data = isin_request(isin)
    if 'symbol' in data.keys():
        return data['symbol']
    else:
        return None

In [ ]:
isin_to_ticker_symbol('AU0000066086')

In [ ]:
def isin_to_ticker(isin):
    symbol = isin_to_ticker_symbol(isin)
    if symbol is not None:
        return yf.Ticker(symbol)
    else:
        return None

In [ ]:
ticker = isin_to_ticker('AU0000066086')

In [ ]:
ticker.history(period='max')['Open'].plot()

## Handling n stocks

In [ ]:
def pretty_portfolio_description(isins):
    output = {}
    pretty_description = "# Companies\n\n"
    for isin in isins:
        ticker = isin_to_ticker(isin)
        pretty_description += "###" + str(isin)
        if ticker is not None:
            info = ticker.info
            if 'sector' in info.keys():
                pretty_description += ' - Sector: ' + str(info['sector']) + "\n"
            else:
                pretty_description += "\n"
            if 'longBusinessSummary' in info.keys():
                description = info['longBusinessSummary']
                as_lines = [description[i:i + 100] for i in range(0, len(description), 100)]
                for line in as_lines:
                    pretty_description += line + "\n"
        pretty_description += "\n\n"
    return pretty_description

In [ ]:
isins = ['AU0000066086', 'US08862E1091', 'US03783T1034'] # Arbitrarily large list of isins. Scrape ???

In [ ]:
description = pretty_portfolio_description(isins)

In [ ]:
description

In [ ]:
with open('../results/portfolio_description.md', 'w') as f:
    f.write(description)

## Summary & final thoughts

- yfinance is convenient but relies too much on actual API
- Better use "https://query2.finance.yahoo.com/v1/finance/..." combined with requests for more flexibility
- Need a better feeling of how reliable information is and how accessible (e.g. Apple earnings)
- Want to scrape as many long descriptions as possible to get a macro-overview using clusterig, NLP, ...